<a href="https://colab.research.google.com/github/dk-wei/ml-algo-implementation/blob/main/Jaccard_Sim_at_scale_MinHash_LSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/ekzhu/datasketch

我们做data mining的时候，常常需要借助**Jaccard Similarity**来寻找similar documents. 一般的jaccard similarity是O(N)复杂度，也就是linear的复杂度。我们可以通过MinHash的方法来降低成sub-linear复杂度。

注意MinHash方法的结果不是100% accurate的，但是他能保证高jaccard sim的documents容易更露头：**sets with higher Jaccard similarities always have higher probabilities to get returned than sets with lower similarities.**

Notebook来源：https://github.com/dataviral/LSH-Jaccard/blob/master/IMDB%20Movie%20Review%20-%20Jaccard%20Similarity.ipynb

In [2]:
!pip install datasketch

     |████████████████████████████████| 67 kB 4.4 MB/s 


In [3]:
import pandas as pd
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm

In [8]:
# df_purchase = pd.read_csv('purchase data.csv')

from google.colab import drive
drive.mount('/content/drive')

# Load the data, contained in the segmentation data csv file.
GD_PATH = '/content/drive/MyDrive/Colab Notebooks/data/'
df = pd.read_csv(GD_PATH+'imdb_master.csv', encoding='latin1', index_col = 0)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# DATA_PATH = "imdb_master.csv" # https://www.kaggle.com/utathya/imdb-review-dataset
# df = pd.read_csv(DATA_PATH, encoding='latin1', index_col=0)

In [22]:
reviews = dict()
new_content = [i for i in df.review.values][:1000]
for key, value in enumerate(new_content):
    reviews[key] = value

In [41]:
lsh = MinHashLSH(
    threshold= 0.3, # Jaccard similarity threshold
    num_perm=128,   # Number of hash functions
)

In [42]:
for key in tqdm(reviews):
    review_hash = MinHash()
    # Iterate over words and add to the review specific hash
    for word in reviews[key].split(" "):
        review_hash.update(word.encode("utf-8"))
    
    # Now insert the review specific hash to lsh object
    lsh.insert(key, review_hash)

100%|██████████| 1000/1000 [00:04<00:00, 208.86it/s]


In [43]:
def query(lsh, sentence):
    """ builds the hash for the sentence and query's the hash for similar senteces """
    sentence_hash = MinHash()
    for word in sentence.split(" "):
        sentence_hash.update(word.encode("utf-8"))
    
    similar_sentences_ids = lsh.query(sentence_hash)
    return similar_sentences_ids

In [44]:
# Pick a sentence in the dataset to use as query senteces
query_review_id = 2

query_review = reviews[query_review_id] # movie review with key as 1000
print("Query Sentence: \n", query_review)

Query Sentence: 
 First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like clichÃ©'e version of gangsters.<br /><br />The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.<br /><br />Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.<br /><br />The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.<br /><br />The only two black people tha

In [45]:
# Find all sentences in the dataset similar to the query sentence
similar_reviews_ids = query(lsh, query_review)

In [46]:
# Show the similar sentences
for i, review_id in enumerate(similar_reviews_ids):
    if review_id == query_review_id: continue
    print("Similar review {}:\n".format(i))
    print(reviews[review_id])
    print("---------\n")

Similar review 1:

Jeremy Irons and Forrest Whitaker are good actors. But this movie was badly written. First of all, during the hijack scene, Irons sits too comfortably in his chair...he appears to be READING something, and rather calmly too! Perhaps the director shot the actor in between takes? Also, the violence at the hijacking was a big letdown. Slow-mo, bullets flying--how his wife and daughter get killed is just not that interesting and the tension is lost. His grieving afterward wasted another 10 minutes. Then he decided to "get revenge" and talk to all his industry journalist friends and ambassadors (he's a journalist for the stuffy Economist rag) and lo and behold, they actually give him tips on where to find the bad guys! How do they know? But what really made me turn the movie off halfway through was when Irons finds his way into a warehouse where baddies are hanging out--BUT NOT THE BADDIES WHO KILLED HIS WIFE--and blows them away anyway. so he's just a murderer. he gets a